# 四、分布式数据并行训练

# 实现分布式优化器封装

实现了一个**参数分片（sharding）的分布式优化器封装器**，思想上接近 **ZeRO Stage-1 / FSDP 的 optimizer state sharding**

## 一、整体功能概览

`ShardedOptimizer` 的目标是：

> **在数据并行（DDP）场景下，让不同 rank 只负责一部分参数的 optimizer state 和更新计算，从而节省显存。**

核心思想是对于**参数本身，每个 rank 都有完整模型参数**，对于**优化器状态（momentum / Adam 的 exp_avg 等），只在参数 owner rank 上存在**， **参数更新只在 owner rank 上进行**。
* **参数同步：更新后通过 `broadcast` 把新参数发给其他 rank**。

## 二、整体架构

```
所有 rank：
  拥有完整模型参数
       │
       ▼
参数被 round-robin 分配 owner_rank
       │
       ▼
每个 rank：
  ├─ 全局 Optimizer（只用于 param_groups 管理）
  └─ 本地 optim（只包含自己拥有的 params）
       │
       ▼
step():
  1. 本地 optim.step()（只更新自己那一片）
  2. dist.broadcast（把更新后的参数同步给所有 rank）
```

---

## 三、代码解读


### 1。 初始化：`__init__`

```python
class ShardedOptimizer(Optimizer):
```

继承 PyTorch 原生 `Optimizer`，因此：

这样可以被 Trainer / 训练循环无缝使用，并且拥有 `param_groups / zero_grad / step` 接口

---

#### 1.1 记录原始 optimizer 类型

```python
self.optimizer_cls = optimizer_cls
self.optimizer_kwargs = kwargs
```

例如：

```python
ShardedOptimizer(
    model.parameters(),
    torch.optim.Adam,
    lr=1e-4
)
```

---

#### 1.2 获取分布式信息

```python
self.rank = dist.get_rank()
self.world_size = dist.get_world_size()
```

用于：

* 决定参数 owner
* 决定 broadcast 源

---

#### 1.3 参数分配辅助变量

```python
self.global_param_counter = 0
self.local_param_groups = []
```

* `global_param_counter`：**全局参数编号**
* 用 round-robin 把参数均匀分配给各个 rank

---

#### 1.4 调用父类构造函数（关键）

```python
super().__init__(params, defaults=kwargs)
```


`Optimizer.__init__` 内部会调用 `self.add_param_group`，但这里的 `add_param_group` 已经被 **重写**

这使得 **参数分片逻辑嵌入在初始化阶段**

---

#### 1.5 创建“本地 optimizer”

```python
self.optim = self.optimizer_cls(self.local_param_groups, **self.optimizer_kwargs)
```

`self.optim` **只包含当前 rank 拥有的参数**，只有这些参数才会分配 optimizer state（节省显存）。

---

### 2. 参数分片逻辑：`add_param_group`

这是整个实现的**核心函数**。

---

#### 2.1 给每个参数打 owner 标记

```python
p._owner_rank = self.global_param_counter % self.world_size
```

round-robin 分配, `_owner_rank` 是动态属性（hack，但可行）

示例（world_size=4）：

| 参数编号 | owner_rank |
| ---- | ---------- |
| p0   | 0          |
| p1   | 1          |
| p2   | 2          |
| p3   | 3          |
| p4   | 0          |
| p5   | 1          |

---

#### 2.2 加入“全局 optimizer”

```python
super().add_param_group(param_group)
```

**为什么必须这么做？**

`self.param_groups` 需要包含 **所有参数**,用于进行`zero_grad`，遍历所有参数做 `broadcast`，并且Trainer / AMP 兼容，**注意**：这个“全局 optimizer”**不做 step**。

---

#### 2.3 构造本地 param group

```python
local_group['params'] = [
    p for p in param_group['params']
    if p._owner_rank == self.rank
]
```

它只保留属于当前 rank 的参数，其他超参（lr、weight_decay）保持一致。

---

#### 2.4 加入本地 optimizer

```python
self.optim.add_param_group(local_group)
```

或者（初始化期间）缓存起来：

```python
self.local_param_groups.append(local_group)
```

---

### 3. 参数更新逻辑：`step`

```python
def step(self, closure=None, **kwargs):
```

---

#### Step 1：本地更新

```python
self.optim.step(**kwargs)
```

* **只更新自己拥有的参数**
* 只有这些参数有 optimizer state

---

#### Step 2：参数同步（核心）

```python
dist.broadcast(p.data, src=p._owner_rank)
```

如果是 owner 则发送最新参数，如果不是 owner 就接收参数覆盖本地副本。最终所有 rank 的模型参数 **完全一致**。

---

### 4.梯度清零：`zero_grad`

```python
super().zero_grad(set_to_none=set_to_none)
```

清除所有参数的梯度，即使不是 owner，也要清 grad（否则梯度会累积）。

---

## 四、这个实现“像什么”？

| 技术           | 相似度  | 区别                        |
| ------------ | ---- | ------------------------- |
| ZeRO Stage-1 | ⭐⭐⭐⭐ | 真实 ZeRO 会用 reduce-scatter |
| FSDP         | ⭐⭐   | 没有参数 flatten / overlap    |
| DDP          | ⭐    | DDP 是全量 optimizer state   |

---

## 五、优点与限制

###  优点

1. **显存节省**

   * optimizer state 约减少到 `1/world_size`
2. **逻辑清晰**

   * 教学 & 原理验证非常好
3. **兼容任意 Optimizer**

   * Adam / SGD / Adafactor 等

---

### 局限

1. **通信成本高**

   ```python
   dist.broadcast(p.data)
   ```

   * 每个参数一次 broadcast（非常慢）

2. **参数粒度过细**

   * 实际系统会 **flatten 后按 bucket 同步**

3. **没有 overlap**

   * 计算与通信完全串行

4. **依赖 monkey-patch 参数属性**

   * `_owner_rank` 不够“干净”

---

## 六、总结


> 用 round-robin 分配参数 owner，
> 只在 owner rank 上更新参数，
> 再用 broadcast 同步权重，
> 从而显著降低 optimizer 显存占用。



In [ ]:
import torch
import torch.distributed as dist
from torch.optim import Optimizer
from typing import Any, Type, Dict, Iterable


class TeachingShardedOptimizer(Optimizer):
    """
    教学版：Optimizer State Sharding（类似 ZeRO Stage-1）

    核心思想：
    - 每个 rank 拥有【完整模型参数】
    - 每个参数被分配一个唯一的 owner rank
    - 只有 owner rank：
        - 持有该参数的 optimizer state
        - 负责执行 optimizer.step()
    - 参数更新后，通过 dist.broadcast 同步到所有 rank
    """

    def __init__(
        self,
        params: Iterable,
        base_optimizer_cls: Type[Optimizer],
        **base_optimizer_kwargs: Any
    ):
        """
        参数说明：
        - params: model.parameters()
        - base_optimizer_cls: 如 torch.optim.Adam
        - base_optimizer_kwargs: 如 lr, betas, weight_decay 等
        """

        # ===== 1. 保存原始 Optimizer 信息 =====
        self.base_optimizer_cls = base_optimizer_cls
        self.base_optimizer_kwargs = base_optimizer_kwargs

        # ===== 2. 分布式环境信息 =====
        self.rank = dist.get_rank()
        self.world_size = dist.get_world_size()

        # ===== 3. 参数分片辅助变量 =====
        # 用于给参数做 round-robin 编号
        self.global_param_index = 0

        # 在 __init__ 期间暂存属于本 rank 的 param groups
        self._buffered_local_param_groups = []

        # ===== 4. 初始化“全局 Optimizer”父类 =====
        # 注意：Optimizer.__init__ 内部会调用 self.add_param_group
        #       而我们在下面重写了 add_param_group
        super().__init__(params, defaults=base_optimizer_kwargs)

        # ===== 5. 创建“本地 shard Optimizer” =====
        # 只包含当前 rank 拥有的参数
        if self._buffered_local_param_groups:
            self.local_shard_optimizer = self.base_optimizer_cls(
                self._buffered_local_param_groups,
                **self.base_optimizer_kwargs
            )
        else:
            # 边缘情况：当前 rank 没有分到任何参数
            self.local_shard_optimizer = self.base_optimizer_cls(
                [],
                **self.base_optimizer_kwargs
            )

    def add_param_group(self, param_group: Dict[str, Any]) -> None:
        """
        教学重点函数：参数分片逻辑发生在这里

        一个 param_group 会被“拆成两份”：
        1. 全量 param_group → 交给父类 Optimizer（用于遍历 & 同步）
        2. 本地 param_group → 只保留 owner == 当前 rank 的参数
        """

        # -------------------------------------------------------
        # Step A: 给 param_group 中的每个参数分配 owner rank
        # -------------------------------------------------------
        for param in param_group["params"]:
            if not hasattr(param, "_owner_rank"):
                # round-robin 分配
                param._owner_rank = self.global_param_index % self.world_size
                self.global_param_index += 1

        # -------------------------------------------------------
        # Step B: 注册到“全局 Optimizer”
        # -------------------------------------------------------
        # 这个 Optimizer：
        # - 包含所有参数
        # - 不负责 step
        # - 只用于：
        #     * zero_grad
        #     * 参数遍历
        #     * step 后的 broadcast
        super().add_param_group(param_group)

        # -------------------------------------------------------
        # Step C: 构造“本地 shard param_group”
        # -------------------------------------------------------
        # 拷贝除 params 以外的超参（lr / weight_decay 等）
        local_param_group = {
            key: value
            for key, value in param_group.items()
            if key != "params"
        }

        # 只保留 owner 是当前 rank 的参数
        local_param_group["params"] = [
            param
            for param in param_group["params"]
            if param._owner_rank == self.rank
        ]

        # -------------------------------------------------------
        # Step D: 加入本地 shard Optimizer
        # -------------------------------------------------------
        if hasattr(self, "local_shard_optimizer"):
            # 正常情况：__init__ 已结束
            if local_param_group["params"]:
                self.local_shard_optimizer.add_param_group(local_param_group)
        else:
            # __init__ 过程中，先缓存
            self._buffered_local_param_groups.append(local_param_group)

    def step(self, closure=None, **kwargs):
        """
        一个 step 分为两步：

        1. 本地更新（只更新 owner 是当前 rank 的参数）
        2. 参数同步（broadcast 最新参数到所有 rank）
        """

        loss = None
        if closure is not None:
            loss = closure()

        # -------------------------------------------------------
        # Step 1: 本地 shard Optimizer 更新
        # -------------------------------------------------------
        # 只有 owner rank 会真正更新对应参数
        self.local_shard_optimizer.step(**kwargs)

        # -------------------------------------------------------
        # Step 2: 参数同步（broadcast）
        # -------------------------------------------------------
        # 遍历“全局 Optimizer”中的所有参数
        for param_group in self.param_groups:
            for param in param_group["params"]:
                # owner rank 作为 src
                # 其他 rank 接收并覆盖本地参数
                dist.broadcast(
                    tensor=param.data,
                    src=param._owner_rank
                )

        return loss

    def zero_grad(self, set_to_none: bool = True):
        """
        梯度必须在所有 rank 上清零：
        即使该 rank 不是参数 owner，也会参与反向传播
        """
        super().zero_grad(set_to_none=set_to_none)
